<a href="https://colab.research.google.com/github/carneiro-fernando/EBAC/blob/main/Projetos/An%C3%A1lise_de_Dados_Interativa_COVID_19_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/coronavirus_banner.jpg?raw=true" alt="covid-banner">

---

# Análise de Dados: COVID-19 Dashboard

---

## Sumário

O notebook está organizado como segue:

1. [Introdução](#intro) - Introdução ao problema e às fontes usadas
2. [Análise Exploratória de Dados.](#data-analysis) - Extração, transformação e carregamento (ETL) e uma breve análise dos dados.
3. [Visualização Interativa de Dados.](#visualization) - Análise interativa dos dados.
4. [Storytelling](#storytelling) - Insights por meio de narrativas


---

### *TLDR*

 - **Dashboard**:  Google Data Studio ([link](https://lookerstudio.google.com/reporting/4a54b790-0672-486e-81fd-037fc2506a06)).
 - **Processamento**: Kaggle Notebook ([link](https://www.kaggle.com/code/carneirofernando/an-lise-de-dados-e-dashboard-impacto-do-covid19)).
 -  **Fonte de dados**: Universidade de Oxford ([link](https://github.com/owid/covid-19-data/tree/master/public/data)).

<a id='intro'></a>

## 1. Introdução

### 1.1 Pandemia do COVID 19

> A pandemia de COVID-19 é uma pandemia global de doença causada pelo coronavírus SARS-CoV-2. O vírus foi identificado pela primeira vez na cidade de Wuhan, na China, em dezembro de 2019. Desde então, o vírus se espalhou para mais de 200 países e territórios, infectando mais de 500 milhões de pessoas e matando mais de 5 milhões.
Fonte: [OMS](https://www.who.int/health-topics/coronavirus)

### 1.2 Fonte dos dados

**Vacinação da COVID-19** - Os dados são compilados pelo projeto Nosso Mundo em Dados (*Our World in Data* ou OWID) da universidade britânica de *Oxford* ([link](https://ourworldindata.org)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de países. O repositório do projeto pode ser acessado neste [link](https://github.com/owid/covid-19-data/tree/master/public/data) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv).

### 1.3 Glossário

> **PPC** - O Produto Interno Bruto com Paridade de Poder de Compra (PPC) é uma medida econômica que leva em conta a diferença no custo de vida e os preços entre os países. Ele ajusta o Produto Interno Bruto (PIB) de diferentes nações para uma base comum, permitindo uma comparação mais precisa do tamanho e da força econômica de cada país, sem ser distorcido pelas variações nos preços locais.
https://www.who.int/data/nutrition/nlis/info/gross-domestic-product-(gdp)-per-capita-and-gdp-per-capita-annual-growth-rate​

> **IDH** - O Índice de Desenvolvimento Humano (IDH) compara indicadores de países nos itens riqueza, alfabetização, educação, esperança de vida, natalidade e outros, com o intuito de avaliar o bem-estar de uma população, especialmente das crianças. Varia de zero a um e é divulgado pelo Programa das Nações Unidas para o Desenvolvimento (Pnud) em seu relatório anual. https://www.undp.org/pt/brazil/o-que-%C3%A9-o-idh


---
<a id='data-analysis'></a>

## 2. Análise Exploratória dos Dados

In [13]:
# Pacotes

import numpy as np
import pandas as pd
import seaborn as sb

In [14]:
# Fonte dos dados
URL = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'

### 2.1 Extração

In [15]:
# Declaração da variável com os dados brutos
data = pd.read_csv(URL, parse_dates=[3])

In [16]:
# Demonstração das colunas disponíveis no dataset
data.columns.values

array(['iso_code', 'continent', 'location', 'date', 'total_cases',
       'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate',
       'icu_patients', 'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units',
       'total_vaccinations', 'people_vaccinated',
       'people_fully_vaccinated', 'total_boosters', 'new_vaccinations',
       'new_vaccinations_smoothed', 't

### 2.2 Wrangling

* **Delimitação de interesse:** para a análise usaremos as seguintes colunas:
  *   Data
  *   País
  *   PPC
  *   IDH
  *   Total casos
  *   Total de mortes
  *   Novas mortes
  *   Novos casos por milhão
  *   Novas mortes por milhão

In [17]:
# Escolhendo as colunas de interesse em um DataFrame chamado covid
covid = data[['date', 'location', 'gdp_per_capita', 'human_development_index','total_cases', 'total_deaths', 'new_deaths', 'new_cases_per_million', 'new_deaths_per_million']]

* **Delimitação temporal:** usaremos o período entre **Março de 2020** e **Março de 2022** para a análise.

In [18]:
# Definindo o índice como temporal para manipulação por tempo
covid = covid.set_index(keys=['date'])

In [19]:
# Restringindo os dados dentro do período de Março de 2020 a Março de 2022
covid = covid['2020-03-31': '2022-03-31']

* **Delimitação espacial:** usaremos dois países com alto IDH (acima de 0.9) e alto PPC (acima de 50000): **Noruega** e **Estados Unidos**. E usaremos dois países com IDH mediano (~ 0.7) e PPC baixo (abaixo de 15000): **Índia** e **Brasil**.

In [20]:
# Delimitando aos 4 países escolhidos
covid = covid.query('location in ["Germany", "United States", "India", "Brazil"]')

In [21]:
# Vamos traduzir os nomes dos países para o Português
paises_dic = {
    "Germany": "Alemanha",
    "United States": "Estados Unidos",
    "India": "Índia",
    "Brazil": "Brasil"
}
covid["location"] = covid["location"].map(paises_dic)

<ipython-input-21-5ec6380a2a9a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid["location"] = covid["location"].map(paises_dic)


Tratando dados faltantes

In [22]:
# Preenchendo valores nulos com o último valor válido
covid = covid.fillna(method='ffill')

In [23]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2924 entries, 2020-03-31 to 2022-03-31
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   location                 2924 non-null   object 
 1   gdp_per_capita           2924 non-null   float64
 2   human_development_index  2924 non-null   float64
 3   total_cases              2924 non-null   float64
 4   total_deaths             2924 non-null   float64
 5   new_deaths               2924 non-null   float64
 6   new_cases_per_million    2924 non-null   float64
 7   new_deaths_per_million   2924 non-null   float64
dtypes: float64(7), object(1)
memory usage: 205.6+ KB


Adicionando porcentagem de mortes por infecções

In [24]:
covid["deaths_percent"] = (covid["total_deaths"]/covid["total_cases"]).round(4)

Visualizando

In [25]:
covid.loc['2021'].drop_duplicates('location')

,location,gdp_per_capita,human_development_index,total_cases,total_deaths,new_deaths,new_cases_per_million,new_deaths_per_million,deaths_percent
date,,,,,,,,,
2021-01-01,Brasil,14103.452,0.765,7619200.0,193875.0,1194.0,258.456,5.545,0.0254
2021-01-01,Alemanha,45229.245,0.947,1660178.0,47009.0,0.0,0.000,0.000,0.0283
2021-01-01,Índia,6426.674,0.645,10286709.0,148994.0,256.0,14.137,0.181,0.0145
2021-01-01,Estados Unidos,54225.446,0.926,19813403.0,355767.0,3763.0,697.089,11.124,0.0180


### 2.3 Carregamento

In [26]:
covid.to_csv('./covid.csv', sep=',', index=True)

---
<a id='visualization'></a>

## 3\. Exploração Interativa de Dados

### 3.1. KPIs

O dashboard de dados contem os seguintes indicadores chaves de desempenho (*key performance indicator* ou KPI) consolidados:

 * Total casos
 * Total de mortes
 * Média de casos por milhão
 * Média de mortes por milhão
 * Porcentagem de mortes em relação aos contaminados
 * Soma de casos no período selecionado

### **3.2. EDA**

O dashboard de dados contem os seguintes gráficos para a análise exploratória de dados (*exploratory data analysis*
ou EDA) interativa:

* Número de mortes por milhão de pessoas
* Porcentagem de mortes por infecções

---
<a id='storytelling'></a>

## 4\. Storytelling

O relatório analisou o impacto da pandemia de COVID-19 durante o período de 2020 a 2022 em quatro países - Brasil, Alemanha, Índia e Estados Unidos - em relação ao Índice de Desenvolvimento Humano (IDH) e o Produto Interno Bruto com paridade de Poder de Compra (PPC). Conclui-se que o IDH e o PPC não têm uma relação direta com o número de mortes por milhão de habitantes causadas pelo vírus. A resposta à pandemia varia de acordo com fatores específicos de cada país, como ações governamentais, medidas de contenção e inúmeros outros fatores. Uma abordagem diferente talvez seja mais eficaz em auxiliar no preparo estratégico para futuras crises de saúde.